In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [2]:
import folium
import geopy

In [25]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


## Getting latitude and longitude of ahmedabd city

In [4]:
from geopy.geocoders import Nominatim

In [5]:
address = "Ahmedabad"
geolocator = Nominatim(user_agent = "ahmd_explorer")

In [6]:
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(" the cordinates for ahmedabad are {} and {}".format(latitude,longitude))

 the cordinates for ahmedabad are 23.0216238 and 72.5797068


In [7]:
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        address,
        LIMIT,
        "4bf58dd8d48988d120951735") # PIZZA PLACE CATEGORY ID
url

'https://api.foursquare.com/v2/venues/explore?&client_id=0P1ZMQHY2O5BKAHBYFCBFET23PJTII41ICBRDWGGNFMAGCUN&client_secret=VS4GJ3XGDGOQECYCFWVXPLY3BAALDBPEDK0NGT0IHPGCHP3W&v=20180605&near=Ahmedabad&limit=100&categoryId=4bf58dd8d48988d120951735'

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e6bb791963d29001bb99b87'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'geocode': {'what': '',
   'where': 'ahmedabad',
   'center': {'lat': 23.02579, 'lng': 72.58727},
   'displayString': 'Ahmedabad, Gujarat, India',
   'cc': 'IN',
   'geometry': {'bounds': {'ne': {'lat': 23.167063007051212,
      'lng': 72.71177811117003},
     'sw': {'lat': 22.934192744971654, 'lng': 72.4286040158387}}},
   'slug': 'ahmadābād',
   'longId': '72057594039207169'},
  'headerLocation': 'Ahmedabad',
  'headerFullLocation': 'Ahmedabad',
  'headerLocationGranularity': 'city',
  'query': 'food court',
  'totalResults': 18,
  'suggestedBounds': {'ne': {'lat': 23.073919431722874,
    'lng': 72.62577429663794},
   'sw': {'lat': 23.01737494070272, 'lng': 72.48833474777915}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': 

In [9]:
food_courts = results['response']['groups'][0]['items']
import json
from pandas.io.json import json_normalize
food_courts = json_normalize(food_courts)

venue_columns = ['venue.name', 'venue.location.address','venue.location.lat','venue.location.lng']
food_courts = food_courts.loc[:,venue_columns]

food_courts.head()

food_courts = food_courts.rename(columns={'venue.name': 'name','venue.location.address': 'address','venue.location.lat': 'lat','venue.location.lng': 'lng'})

In [24]:
food_courts


,name,address,lat,lng
0,Food court @ alpha1,Alpha 1,23.039964,72.531746
1,Ankur Crossroad,Ankur rd,23.058802,72.555842
2,Ahmedabad One,Vastrapur,23.039758,72.531317
3,Subway,Domestic airport,23.071349,72.619527
4,Municipal Market Food Stalls,"C G Road, Navrangpura.",23.034432,72.560930
5,global Desi tadka,Gulmohar Park Mall,23.026803,72.509855
6,Mevad Dabeli,Vijay Cross Road,23.040800,72.553066
7,Friends Maggi,NaN,23.038465,72.554508
8,Blues,NaN,23.030488,72.559229
9,"Amul parlour, Nr Satyam Mall",Opp Kameshwar school,23.031289,72.528250


In [10]:
map_ahmd = folium.Map([latitude,longitude],min_zoom = 11, tiles = 'OpenStreetMap', color = 'yellow', opacity = 0.4)



In [11]:
places = folium.map.FeatureGroup()
places

In [12]:

for lat, lng, in zip(food_courts.lat, food_courts.lng):
        places.add_child(
        folium.CircleMarker(
        [lat,lng],
        radius = 3,
        color = 'red',
        fill = True,
        fill_color = 'blue',
        icon=folium.Icon(color='green'),
        fill_opacity = 0.6
        )
    )
latitudes = list(food_courts.lat)
longitudes = list(food_courts.lng)
labels = list(food_courts.name)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(map_ahmd)    
    
# add incidents to map
map_ahmd.add_child(places)

In [13]:
LIMIT = 100

url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        address,
        LIMIT,
        "4d4b7105d754a06372d81259") # PIZZA PLACE CATEGORY ID
url1

'https://api.foursquare.com/v2/venues/explore?&client_id=0P1ZMQHY2O5BKAHBYFCBFET23PJTII41ICBRDWGGNFMAGCUN&client_secret=VS4GJ3XGDGOQECYCFWVXPLY3BAALDBPEDK0NGT0IHPGCHP3W&v=20180605&near=Ahmedabad&limit=100&categoryId=4d4b7105d754a06372d81259'

In [14]:
results1 = requests.get(url1).json()
results1

{'meta': {'code': 200, 'requestId': '5e6bb8c577af03001bfddb9a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'geocode': {'what': '',
   'where': 'ahmedabad',
   'center': {'lat': 23.02579, 'lng': 72.58727},
   'displayString': 'Ahmedabad, Gujarat, India',
   'cc': 'IN',
   'geometry': {'bounds': {'ne': {'lat': 23.167063007051212,
      'lng': 72.71177811117003},
     'sw': {'lat': 22.934192744971654, 'lng': 72.4286040158387}}},
   'slug': 'ahmadābād',
   'longId': '72057594039207169'},
  'headerLocation': 'Ahmedabad',
  'headerFullLocation': 'Ahmedabad',
  'headerLocationGranularity': 'city',
  'query': 'college education',
  'totalResults': 36,
  'suggestedBounds': {'ne': {'lat': 23.16342560295848,
    'lng': 72.6779139666385},
   'sw': {'lat': 22.98714148893784, 'lng': 72.43125476292377}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'ite

In [15]:
colleges = results1['response']['groups'][0]['items']

colleges = json_normalize(colleges)

colleges_columns = ['venue.name', 'venue.location.address','venue.location.lat','venue.location.lng']
college = colleges.loc[:,colleges_columns]

college.head()



,venue.name,venue.location.address,venue.location.lat,venue.location.lng
0,Polytechnic,NaN,23.027220,72.545060
1,Entrepreneurship Development Institute of India,"(Via Ahmedabad Airport & Indira Bridge),",23.114770,72.629205
2,National Institute of Design (NID),Bhagtacharya Road,23.011295,72.569239
3,St. Xaviers College,Navrangpura,23.033999,72.553295
4,CEPT University,NaN,23.037340,72.549918


In [16]:

college = college.rename(columns={'venue.name': 'name','venue.location.address': 'address','venue.location.lat': 'latt','venue.location.lng': 'lngg'})

college

,name,address,latt,lngg
0,Polytechnic,NaN,23.027220,72.545060
1,Entrepreneurship Development Institute of India,"(Via Ahmedabad Airport & Indira Bridge),",23.114770,72.629205
2,National Institute of Design (NID),Bhagtacharya Road,23.011295,72.569239
3,St. Xaviers College,Navrangpura,23.033999,72.553295
4,CEPT University,NaN,23.037340,72.549918
5,HL institute of commerce,NaN,23.037450,72.555209
6,AMC MET Medical College,"L.g.hospital,",22.995154,72.615414
7,Narayana Business School,"Narayana House, Nr. Ishan 3, Opp. Shell Petrol...",23.015640,72.511847
8,Arihant Institute Pvt Ltd,"2, Navin Park Society",23.051530,72.562071
9,Vishwakrma Government Engineering College,NaN,23.105780,72.597223


In [17]:
clg = folium.map.FeatureGroup()

In [18]:
for latt, lngg, in zip(college.latt, college.lngg):
        clg.add_child(
        folium.CircleMarker(
        [latt,lngg],
        radius = 3,
        icon = folium.Icon(color='green'),
        fill = False,
        color='green',
        fill_color='green',
        fill_opacity = 0.6
        )
        )
latitudes1 = list(college.latt)
longitudes1 = list(college.lngg)
labels1 = list(college.name)

for lat, lng, label in zip(latitudes1, longitudes1, labels1):
    folium.Marker([lat, lng], popup=label).add_to(map_ahmd)    
    
# add incidents to map
map_ahmd.add_child(clg)


In [19]:
LIMIT = 100

url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        address,
        LIMIT,
        "4bf58dd8d48988d17f941735") #PLACE CATEGORY ID
url2

results2 = requests.get(url2).json()
results2

theatres = results2['response']['groups'][0]['items']

theatres = json_normalize(theatres)

theatres_columns = ['venue.name', 'venue.location.address','venue.location.lat','venue.location.lng']
theatre = theatres.loc[:,colleges_columns]

theatre.head()

theatre = theatre.rename(columns={'venue.name': 'name','venue.location.address': 'address','venue.location.lat': 'latitude','venue.location.lng': 'longitude'})

theatre


,name,address,latitude,longitude
0,PVR Cinemas,Acropolis Mall,23.048943,72.516024
1,Cinepolis,Alpha One Mall,23.039896,72.531727
2,"IMAX 3D, Gujarat Science City","Village Hebatpur, Sola-Santej Road",23.038844,72.497140
3,Sunset Drive-In Theatre,NaN,23.049051,72.528395
4,PVR,"4th Floor,4D Square Mall,",23.103045,72.595351
5,Grand Cinema,Balaji Agora Mall,23.118144,72.624498
6,Ahmedabad One,Vastrapur,23.039758,72.531317
7,Mukta Cinema,Gulmohar Mall,23.027332,72.510164
8,Shiv Cinemax,Ashram road,23.031889,72.570513
9,City Pulse,"10, Acer Mall, Raipur",23.014890,72.596235


In [20]:

movie = folium.map.FeatureGroup()

for latitude, longitude in zip(theatre.latitude, theatre.longitude):
    movie.add_child(
    folium.CircleMarker(
    [latitude, longitude],
    radius = 3,
    fill = True,
    color='yellow',
    fill_opacity = 0.6
    )
)
latitudes2 = list(theatre.latitude)
longitudes2 = list(theatre.longitude)
labels2 = list(theatre.name)

for lat, lng, label in zip(latitudes2, longitudes2, labels2):
    folium.Marker([lat, lng], popup=label, fill_color = 'yellow').add_to(map_ahmd)    
    
# add incidents to map
map_ahmd.add_child(movie)

In [22]:
for lat, lng, in zip(food_courts.lat, food_courts.lng):
    folium.Circle(
        radius=2500,
        location=[lat, lng],
        color='black',
        fill=False,
    ).add_to(map_ahmd)
map_ahmd

### based on the black circles around the existing food courts, we can assume that one food court is targetting customers from neighborhoods of 2.5 km radius. now we have our options as we can clearly see that areas outside of the black circles are our main target. choosing the location which best fits our requirements are 1. chanakyapuri and 2. chandkheda as both have multiple colleges/universities and multiple theatres in the desired radius. that fulfills our goal.